# Imports

In [ ]:
import sys
sys.path.append('..')

import time
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

import os
import json
from time import time
import numpy as np
import scipy as sp
import scipy.sparse as spsp
#from tqdm import tqdm

import time
import theano
import theano.tensor as T
from theano.sandbox.cuda.dnn import dnn_conv

# This sets which GPU to use
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu2")

from PIL import Image
import logging

theano.config.scan.allow_gc = True
theano.config.exception_verbosity = 'low'
theano.config.warn_float64 = 'raise'
theano.config.floatX = 'float32'

# Set this to the current working directory
# (the directory where the gae.ipynb file is stored)
cwd_path = '/home/jsvoboda/Documents/fingerprits/gae/'
# Set this to the folder where your data (synthetic fingerprints) are stored
data_path = '/media/nas/fingerprints/'

logger = logging.getLogger('fingerprints')
hdlr = logging.FileHandler(os.path.join(cwd_path, 'training.log'))
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)

from lib import activations
from lib import updates
from lib import inits
from lib.rng import py_rng, np_rng
from lib.ops import batchnorm, conv_cond_concat, deconv, dropout, l2normalize
from lib.metrics import nnc_score, nnd_score
from lib.theano_utils import floatX, sharedX
from lib.data_utils import OneHot, shuffle, iter_data, center_crop, patch

N.B. The code from the following imports is lifted from the original [dcgan project](https://github.com/Newmu/dcgan_code)

# Data Stuff

In [ ]:
def load_matlab_file(path_file, name_field):
    """
    load '.mat' files
    inputs:
        path_file, string containing the file pathd
        name_field, string containig the field name (default='shape')
    warning:
        '.mat' files should be saved in the '-v7.3' format
    """
    retry = 1
    while retry:
        try:
            db = h5py.File(path_file, 'r')
            ds = db[name_field]
            retry = 0
        except IOError:
            retry = 1
        
    try:
        if 'ir' in ds.keys():
            data = np.asarray(ds['data'])
            ir   = np.asarray(ds['ir'])
            jc   = np.asarray(ds['jc'])
            out  = sp.csc_matrix((data, ir, jc)).astype(np.float32)
    except AttributeError:
        # Transpose in case is a dense matrix because of the row- vs column- major ordering between python and matlab
        out = np.asarray(ds).astype(np.float32).T

    db.close()

    return out

In [ ]:
# train / test splitting
path        = cwd_path
fSamples    = open(path + 'dataset.txt', 'w')
fTargets    = open(path + 'targets.txt', 'w')
fMinutiae   = open(path + 'minutiae.txt', 'w')
#dirs        = range(1, 9)
samples     = range(1, 1000)
#for di in dirs:
for sam in samples:
    fSamples.write('sample_train_bg_8/%d.png\n' % (sam))
    fTargets.write('sample_train_bg_8_targets_bin/%d.png\n' % (sam))
    fMinutiae.write('sample_train_bg_8_targets_minutiae/feats_%d.mat\n' % (sam))
samples     = range(1, 1000)
for sam in samples:
    fSamples.write('sample_test_bg_2/%d.png\n' % (sam))
    fTargets.write('sample_test_bg_2_targets_bin/%d.png\n' % (sam))
    fMinutiae.write('sample_test_bg_2_targets_minutiae/feats_%d.mat\n' % (sam))
fSamples.close()
fTargets.close()
fMinutiae.close()

fEval = open(path + 'eval.txt', 'w')
samples = range(1, 38)
eval_dir = 'eval_data/'
if os.path.exists(eval_dir):
    files = os.listdir(eval_dir)
    for aFile in files:
        if os.path.splitext(aFile)[1] in ['.png', '.jpg', '.bmp']:
            fEval.write(os.path.join(eval_dir, aFile) + '\n')
fEval.close()

In [ ]:
from sklearn.preprocessing import normalize
from skimage import filters
from skimage import data, img_as_float
from skimage import exposure
import skimage.filters as skif
import skimage.morphology as skim
import scipy.ndimage.filters as spif
import skimage.util as skiu

def rgb2gray(rgb):
    return (0.299 * rgb[:, :, 0] + 0.587 * rgb[:, :, 1] + 0.114 * rgb[:, :, 2]).astype(np.uint8)

def flipInPlace(image):
    origSizeX, origSizeY = image.shape[0], image.shape[1]

    flippedImages = {
        0: image, # Original image
        1: np.flipud(image), # Horizontal flip
        2: np.fliplr(image), # Vertical flip
        3: np.flipud(np.fliplr(image)) # Hor & Vert flip
    }

    flipType = np.random.randint(4)

    return flippedImages[flipType]

# dataset
class FaceDataset(object):
    def __init__(self, dataset_txt, targets_txt, minutiae_txt, face_paths, eval_txt, eval_paths, batch_size = 12, epoch_size = 1):
        self.dataset_txt     = dataset_txt
        self.targets_txt     = targets_txt
        self.minutiae_txt    = minutiae_txt
        self.eval_txt        = eval_txt
        self.face_path       = face_paths
        self.eval_path       = eval_paths
        self.epoch_size    = epoch_size
        self.batch_size    = batch_size
        #self.target_dim = (160, 128)
        self.target_dim = (320, 256)
        #self.target_dim = (185, 128)
        self.normalize  = True
        self.threshold = False
        
        self.train_names   = []
        self.train_targets_names = []
        self.train_minutiae_names = []
        self.test_names = []
        self.test_targets_names = []
        self.test_minutiae_names = []
        self.eval_names = []
        
        with open(self.dataset_txt, 'r') as f:
            self.dataset_fnames = [line.rstrip() for line in f]
            
        with open(self.targets_txt, 'r') as f:
            self.targets_fnames = [line.rstrip() for line in f]    
            
        with open(self.minutiae_txt, 'r') as f:
            self.minutiae_fnames = [line.rstrip() for line in f]
            
        with open(self.eval_txt, 'r') as f:
            self.eval_fnames = [line.rstrip() for line in f]
            
        print("Loading dataset ...")
        sys.stdout.flush()
        for i in np.arange(0, len(self.dataset_fnames) - int(len(self.dataset_fnames) - 100)):
            self.train_names.append(os.path.join(self.face_path, self.dataset_fnames[i]))
            self.train_targets_names.append(os.path.join(self.face_path, self.targets_fnames[i]))
            self.train_minutiae_names.append(os.path.join(self.face_path, self.minutiae_fnames[i]))
        for i in np.arange(int(len(self.dataset_fnames) - 100), len(self.dataset_fnames)):
            self.test_names.append(os.path.join(self.face_path, self.dataset_fnames[i]))
            self.test_targets_names.append(os.path.join(self.face_path, self.targets_fnames[i]))
            self.test_minutiae_names.append(os.path.join(self.face_path, self.minutiae_fnames[i]))
        for i in np.arange(0, len(self.eval_fnames)):
            self.eval_names.append(os.path.join(self.eval_path, self.eval_fnames[i]))
            
        self.train_perm_idxs = np.random.permutation(len(self.train_names))
        self.train_iter_cntr = 0
        
        self.fixed_index = None
        
    def enhance_image(self, img):
        #img = skif.gaussian_filter(img, sigma = 0.05)
        #img = exposure.equalize_adapthist(img, clip_limit = 0.03)
        #img = img > 0.6#thresh
        #img = skim.binary_closing(img, np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]]))
        #img = skim.binary_dilation(img, np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]]))        
        #img = skif.gaussian_filter(img, sigma = 0.05)
        return img
    
    def modify_image(self, img):
        img = skiu.random_noise(img, mode='speckle', seed=None, clip=True, mean = 0, var = 0.0035)
        #for i in range(int(round(np.random.rand() * 2))):
            #img = skim.erosion(img)
            
        return img
    
    def train_iter(self):
        for i in xrange(self.epoch_size):
            imgs, targets, masks, minutiaes = [], [], [], []
            for b in xrange(self.batch_size):
                self.train_iter_cntr += 1
                if self.train_iter_cntr == len(self.train_names):
                    self.train_perm_idxs = np.random.permutation(len(self.train_names))
                    self.train_iter_cntr = 0
                #idx = np.random.permutation(len(self.train_names))[0]
                idx = self.train_perm_idxs[self.train_iter_cntr]
                if self.fixed_index != None:
                    idx = self.fixed_index
        
                minutiae = load_matlab_file(self.train_minutiae_names[idx], 'feats_mat').squeeze()
        
                img = rgb2gray(sp.misc.imread(self.train_names[idx]))
                img = sp.misc.imresize(img, self.target_dim) 
                img = img_as_float(img)
                img = self.modify_image(img)
                #img = exposure.equalize_adapthist(img, clip_limit = 0.03)
                
                target = (sp.misc.imread(self.train_targets_names[idx]))
                target = sp.misc.imresize(target, self.target_dim) 
                target = img_as_float(target)
                target = self.enhance_image(target)

                target = target.reshape((1, img.shape[0], img.shape[1]))
                mask = np.zeros(img.shape)
                img = img.reshape((1, img.shape[0], img.shape[1]))
                mask = mask.reshape((1, mask.shape[0], mask.shape[1]))
                minutiae = minutiae.reshape((1, minutiae.shape[0], minutiae.shape[1]))

                imgs.append(img)
                targets.append(target)
                masks.append(mask)
                minutiaes.append(minutiae)

            imgs, targets, masks, minutiaes = np.array(imgs), np.array(targets), np.array(masks), np.array(minutiaes)
            yield imgs, targets, masks, minutiaes
                    
    def train_iter_all(self):
        for idx in xrange(len(self.train_names)):
            minutiae = load_matlab_file(self.train_minutiae_names[idx], 'feats_mat').squeeze()
                
            img = rgb2gray(sp.misc.imread(self.train_names[idx]))
            img = sp.misc.imresize(img, self.target_dim)
            img = img_as_float(img)
            img = self.modify_image(img)
            #img = exposure.equalize_adapthist(img, clip_limit = 0.03)

            target = (sp.misc.imread(self.train_targets_names[idx]))
            target = sp.misc.imresize(target, self.target_dim) 
            target = img_as_float(target)
            target = self.enhance_image(target)

            target = target.reshape((1, 1, img.shape[0], img.shape[1]))
            mask = np.zeros(img.shape)
            img = img.reshape((1, 1, img.shape[0], img.shape[1]))
            mask = mask.reshape((1, 1, mask.shape[0], mask.shape[1]))                
            minutiae = minutiae.reshape((1, minutiae.shape[0], minutiae.shape[1]))

            yield img, target, mask, minutiae
            
    def valid_iter(self):
        idx = 0
            
        minutiae = load_matlab_file(self.train_minutiae_names[idx], 'feats_mat').squeeze()
        
        img = rgb2gray(sp.misc.imread(self.train_names[idx]))
        img = sp.misc.imresize(img, self.target_dim)
        img = img_as_float(img)
        img = self.modify_image(img)
        #img = exposure.equalize_adapthist(img, clip_limit = 0.03)
                
        target = (sp.misc.imread(self.train_targets_names[idx]))
        target = sp.misc.imresize(target, self.target_dim) 
        target = img_as_float(target)
        target = self.enhance_image(target)
                
        target = target.reshape((1, 1, img.shape[0], img.shape[1]))
        mask = np.zeros(img.shape)
        img = img.reshape((1, 1, img.shape[0], img.shape[1]))
        mask = mask.reshape((1, 1, mask.shape[0], mask.shape[1]))                
        minutiae = minutiae.reshape((1, minutiae.shape[0], minutiae.shape[1]))

        yield img, target, mask
        
    def test_iter(self):
        idx = np.random.permutation(len(self.test_names))[0]     
        
        minutiae = load_matlab_file(self.train_minutiae_names[idx], 'feats_mat').squeeze()   
        
        img = rgb2gray(sp.misc.imread(self.test_names[idx]))
        img = sp.misc.imresize(img, self.target_dim)
        img = img_as_float(img)
        img = self.modify_image(img)
        #img = exposure.equalize_adapthist(img, clip_limit = 0.03)
        
        target = (sp.misc.imread(self.test_targets_names[idx]))
        target = sp.misc.imresize(target, self.target_dim) 
        target = img_as_float(target)
        target = self.enhance_image(target)
        
        target = target.reshape((1, 1, img.shape[0], img.shape[1]))
        mask = np.zeros(img.shape)
        img = img.reshape((1, 1, img.shape[0], img.shape[1]))
        mask = mask.reshape((1, 1, mask.shape[0], mask.shape[1]))                
        minutiae = minutiae.reshape((1, minutiae.shape[0], minutiae.shape[1]))

        yield img, target, mask

    def test_iter_all(self):
        for idx in xrange(len(self.test_names) // self.epoch_size):                
            imgs, targets, masks, minutiaes = [], [], [], []
            for i in xrange(self.epoch_size):
                minutiae = load_matlab_file(self.train_minutiae_names[idx], 'feats_mat').squeeze()
            
                img = rgb2gray(sp.misc.imread(self.test_names[idx * self.epoch_size + i]))
                img = sp.misc.imresize(img, self.target_dim)
                img = img_as_float(img)
                img = self.modify_image(img)
                #img = exposure.equalize_adapthist(img, clip_limit = 0.03)
        
                target = (sp.misc.imread(self.test_targets_names[idx * self.epoch_size + i]))
                target = sp.misc.imresize(target, self.target_dim) 
                target = img_as_float(target)
                target = self.enhance_image(target)
                
                target = target.reshape((1, img.shape[0], img.shape[1]))
                mask = np.zeros(img.shape)
                img = img.reshape((1, img.shape[0], img.shape[1]))
                mask = mask.reshape((1, mask.shape[0], mask.shape[1]))                
                minutiae = minutiae.reshape((1, minutiae.shape[0], minutiae.shape[1]))

                imgs.append(img)
                targets.append(target)
                masks.append(mask)
                minutiaes.append(minutiae)

            imgs, targets, masks, minutiaes = np.array(imgs), np.array(targets), np.array(masks), np.array(minutiaes)
            yield imgs, targets, masks, minutiaes
              
    def eval_iter_all(self):
        for idx in xrange(len(self.eval_names) // self.epoch_size + 1):                
            imgs, targets, masks, names = [], [], [], []
            for i in xrange(self.epoch_size):
                if idx * self.epoch_size + i >= len(self.eval_names):
                    break
                    
                #img = rgb2gray(sp.misc.imread(self.eval_names[idx * self.epoch_size + i]))
                img = sp.misc.imread(self.eval_names[idx * self.epoch_size + i])
                img = sp.misc.imresize(img, self.target_dim)
               
                img = img_as_float(img)
                padsz = 60
                padsz2 = padsz // 2
                imgg = np.ones((img.shape[0] + padsz, img.shape[1] + padsz))
                imgg[padsz2:img.shape[0]+padsz2, padsz2:img.shape[1]+padsz2] = img
                img = imgg#sp.misc.imresize(imgg, self.target_dim)
                
                #img = self.modify_image(img)
                img = exposure.equalize_adapthist(img, clip_limit = 0.03)
                #img = img > 0.5
                
                #img = exposure.equalize_adapthist(img, clip_limit = 0.025)
                #img = exposure.equalize_adapthist(img, clip_limit = 0.005)
                
                target = img.copy().reshape((1, img.shape[0], img.shape[1]))
                #target = target > 0.99
                #target = skim.binary_closing(target, np.ones((3, 3)))
                #img, mask = self.random_corrupt3(img)
                #img, mask = self.random_corrupt(img)
                #img, mask = self.random_corrupt3(img)
                mask = np.zeros(img.shape)
                img = img.reshape((1, img.shape[0], img.shape[1]))
                mask = mask.reshape((1, mask.shape[0], mask.shape[1]))

                imgs.append(img)
                targets.append(target)
                masks.append(mask)
                names.append(self.eval_names[idx * self.epoch_size + i])

            imgs, targets, masks = np.array(imgs), np.array(targets), np.array(masks)            
            yield imgs, targets, masks, names
            
tic = time.time()
ds = FaceDataset(os.path.join(cwd_path, 'dataset.txt'), 
                 os.path.join(cwd_path, 'targets.txt'),
                 os.path.join(cwd_path, 'minutiae.txt'),
                  os.path.join(data_path, 'fingerprints_newdata/'),
                  os.path.join(cwd_path, 'eval.txt'),
                  os.path.join(data_path, 'fingerprints_eval/'))
print('Loading done! (duration: {0} seconds)'.format(time.time() - tic))

In [ ]:
# Code for debugging purposes, for easily visualizing the data loaded by the dataset
'''
for te_train, te_target, te_mask, names in ds.eval_iter_all():
    te_out, te_ta = ae_encode(input_transform(te_train), target_transform(te_target))
    te_out, te_ta = ae_encode(te_out, te_ta)
    te_out, te_ta = ae_encode(te_out, te_ta)
    te_out, te_ta = ae_encode(te_out, te_ta)
    #te_out, te_ta = ae_encode(te_out, te_ta)
    #te_out, te_ta = ae_encode(te_out, te_ta)

    te_reshape = inverse(te_out)
    te_target_reshape = inverse(te_ta)
    te_train_reshape = inverse(input_transform(te_train))
    
    plt.figure()
    plt.imshow(te_train[0, 0, :, :], 'gray')
    
    plt.figure()
    plt.imshow(te_reshape[0, 0, :, :], 'gray')

    plt.figure()
    plt.imshow(te_target_reshape[0, 0, :, :], 'gray')
    
    #diffimg = np.abs(te_reshape[0, 0, :, :] - te_target_reshape[0, 0, :, :])
    #plt.figure()
    #plt.imshow(diffimg)
    
    #print np.min(diffimg.flatten())
    #print np.max(diffimg.flatten())
    #print np.mean(diffimg.flatten())
    
    break
'''

# Setup Neural Network

In [ ]:
def target_transform(X):
    return floatX(X).transpose(0, 1, 3, 2)

def input_transform(X):
    return target_transform(X)

def mask_transform(X):
    return X.transpose(0, 1, 3, 2)

In [ ]:
l2 = 1e-4        # l2 weight decay
b1 = 0.5          # momentum term of adam
nc = 1#3            # # of channels in image
lr = 0.0002       # initial learning rate for adam

relu = activations.Rectify()
sigmoid = activations.Sigmoid()
lrelu = activations.LeakyRectify()
tanh = activations.Tanh()
bce = T.nnet.binary_crossentropy

def wmse(x,y, weights):
    return T.sum(T.pow(x-y,2) * weights, axis = 1)

def mse(x,y):
    return T.sum(T.pow(x-y,2), axis = 1)

def mmse(x,y,m):
    return T.sum(T.pow(x-y,2)*m, axis = 1) / T.sum(T.gt(m, 1e-6), axis = 1, dtype = theano.config.floatX)

gifn = inits.Normal(scale=0.02)
difn = inits.Normal(scale=0.02)
sigma_ifn = inits.Normal(loc = -100., scale=0.02)
gain_ifn = inits.Normal(loc=1., scale=0.02)
bias_ifn = inits.Constant(c=0.)

In [ ]:
'''
The following methods are to help adjust the sizes of the convolutional layers in the generator and discriminator, 
which is very fiddly to do otherwise.  The (overloaded) method make_conv_set can be used to create both the conv 
and deconv sets of layers.  Note that the 'size' of the images is the size of the shortest side (32 in the input set, 
128 in the target set).  Only use powers of 2 here.
'''
def make_conv_layer(X, input_size, output_size, input_filters, 
                    output_filters, name, index,
                    weights = None, filter_sz = 5):
    
    is_deconv = output_size >= input_size

    w_size = (input_filters, output_filters, filter_sz, filter_sz) \
            if is_deconv else (output_filters, input_filters, filter_sz, filter_sz)
    
    if weights is None:
        w = gifn(w_size, '%sw%i' %(name, index))
        g = gain_ifn((output_filters), '%sg%i' %(name, index))
        b = bias_ifn((output_filters), '%sb%i' %(name, index))
    else:
        w,g,b = weights
        
    print w_size
    
    conv_method = deconv if is_deconv else dnn_conv
    activation = relu if is_deconv else lrelu
        
    sub = output_size / input_size if is_deconv else input_size / output_size
    
    bm = filter_sz // 2
    #if filter_sz == 3:
    #    bm = 1
    #else:
    #    bm = 2
    
    layer = activation(batchnorm(conv_method(X, w, subsample=(sub, sub), border_mode=(bm, bm)), g=g, b=b))
    
    return layer, [w,g,b]

def make_conv_set(input, layer_sizes, num_filters, name, weights = None, filter_szs = None):
    assert(len(layer_sizes) == len(num_filters))
    
    vars_ = []
    layers_ = []
    current_layer = input
    
    for i in range(len(layer_sizes) - 1):
        input_size = layer_sizes[i]
        output_size = layer_sizes[i + 1]
        input_filters = num_filters[i]
        output_filters = num_filters[i + 1]
        
        if weights is not None:
            this_wts = weights[i * 3 : i * 3 + 3]
        else:
            this_wts = None
            
        if filter_szs != None:
            filter_sz = filter_szs[i]
        else:
            filter_sz = 5
        
        layer, new_vars = make_conv_layer(current_layer, input_size, output_size, 
                                          input_filters, output_filters, name, i, 
                                          weights = this_wts, filter_sz = filter_sz)
        
        vars_ += new_vars
        layers_ += [layer]
        current_layer = layer
        
    return current_layer, vars_, layers_

In [ ]:
import pickle
####
# Use code below if you want use a saved model
####
'''
[e_params, g_params] = pickle.load( open( "model_params_350.pkl", "rb" ) )
gwx = g_params[-1]

# inputs
X = T.tensor4()
# masks
minutiae = T.tensor4()

## encode layer
e_layer_sizes = [256, 128, 64, 32, 16, 8]
e_filter_sizes = [1, 128, 256, 256, 512, 1024]
e_fszs = [13, 11, 9, 7, 5]

eX, e_params, e_layers = make_conv_set(X, e_layer_sizes, e_filter_sizes, "e", weights=e_params, filter_szs = e_fszs)

## generative layer
g_layer_sizes = [8, 16, 32, 64, 128, 256]
g_num_filters = [1024, 512, 256, 256, 128, 128]
g_fszs = [5, 7, 9, 11, 13]

g_out, g_params, g_layers = make_conv_set(eX, g_layer_sizes, g_num_filters, "g", weights=g_params, filter_szs = g_fszs)
g_params += [gwx]
gX = sigmoid(deconv(g_out, gwx, subsample=(1, 1), border_mode=(2, 2)))

# target outputs
target = T.tensor4()
'''

####
#Use code below if you are training a model from scratch
####
# inputs
X = T.tensor4()
# masks
minutiae = T.tensor4()

## encode layer
e_layer_sizes = [256, 128, 64, 32, 16, 8]
e_filter_sizes = [1, 128, 256, 256, 512, 1024]
e_fszs = [13, 11, 9, 7, 5]

eX, e_params, e_layers = make_conv_set(X, e_layer_sizes, e_filter_sizes, "e", filter_szs = e_fszs)

## generative layer
g_layer_sizes = [8, 16, 32, 64, 128, 256]
g_num_filters = [1024, 512, 256, 256, 128, 128]
g_fszs = [5, 7, 9, 11, 13]

g_out, g_params, g_layers = make_conv_set(eX, g_layer_sizes, g_num_filters, "g", filter_szs = g_fszs)
gwx = gifn((128, nc, 5, 5), 'gwx')
g_params += [gwx]
gX = sigmoid(deconv(g_out, gwx, subsample=(1, 1), border_mode=(2, 2)))

# target outputs
target = T.tensor4()

In [ ]:
import numpy as np
import scipy.ndimage.filters as fi

def gkern2(kernlen=21, nsig=3):
    """Returns a 2D Gaussian kernel array."""

    # create nxn zeros
    inp = np.zeros((kernlen, kernlen))
    # set element at the middle to one, a dirac delta
    inp[kernlen//2, kernlen//2] = 1
    # gaussian-smooth the dirac, resulting in a gaussian filter mask
    return fi.gaussian_filter(inp, nsig)

def ridge_orientation(img):
    # Compute ridge orientations
    # Precomputed gaussian
    gauss_block = theano.shared(np.array([[gkern2(7, 1)]],
                dtype = theano.config.floatX), name ='gauss_block')
    gauss_orient = theano.shared(np.array([[gkern2(31, 5)]],
                dtype = theano.config.floatX), name ='gauss_orient')
    dx = theano.shared(np.array([[[[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]]],
                dtype = theano.config.floatX), name ='dx')
    dy = theano.shared(np.array([[[[-1, -1, -1], [0, 0, 0], [1, 1, 1]]]],
            dtype = theano.config.floatX), name ='dy')
    # Get gaussian gradient
    gfx = T.nnet.conv2d(gauss_block, dx, border_mode = 'half')
    gfy = T.nnet.conv2d(gauss_block, dy, border_mode = 'half')
    # Get image gradients
    gfx_img = T.nnet.conv2d(img, gfx, border_mode = 'half')
    gfy_img = T.nnet.conv2d(img, gfy, border_mode = 'half')
    # Covariance of the data for image gradients
    gfxx_img, gfxy_img, gfyy_img = gfx_img**2, gfx_img * gfy_img, gfy_img**2

    # Smooth the covariance data
    gfxx_img = T.nnet.conv2d(gfxx_img, gauss_orient, border_mode = 'half')
    gfxy_img = 2 * T.nnet.conv2d(gfxy_img, gauss_orient, border_mode = 'half')
    gfyy_img = T.nnet.conv2d(gfyy_img, gauss_orient, border_mode = 'half')

    # Analytic solution of the principal direction
    denom = T.sqrt(gfxy_img**2 + (gfxx_img - gfyy_img)**2 + 1e-12)
    sin2theta = gfxy_img / denom
    cos2theta = (gfxx_img - gfyy_img) / denom

    # Smooth the double angles
    sin2theta = T.nnet.conv2d(sin2theta, gauss_orient, border_mode = 'half')
    cos2theta = T.nnet.conv2d(cos2theta, gauss_orient, border_mode = 'half')

    # Compute the final orientation
    orient_img = np.pi / 2.0 + T.arctan2(sin2theta, cos2theta) / 2.0
    
    # Reliability and coherence
    Imin = (gfyy_img + gfxx_img) / 2.0 - (gfxx_img - gfyy_img) * cos2theta / 2.0 - gfxy_img * sin2theta / 2.0
    Imax = gfyy_img + gfxx_img - Imin
    
    reliability = 1 - Imin / (Imax + 0.001)
    reliability = T.switch(T.gt(denom, 0.001), reliability, 0)
    coherence = ((Imax - Imin) / (Imax + Imin + 1e-12))**2
    
    return orient_img, reliability, coherence

imgg = T.ftensor4('imgg')
output, rel, coh = ridge_orientation(imgg)

forient = theano.function([imgg],
                            [output],
                            allow_input_downcast = True)

def ridge_gradient(img):
    dx = theano.shared(np.array([[[[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]]],
            dtype = theano.config.floatX), name ='dx')
    dy = theano.shared(np.array([[[[-1, -1, -1], [0, 0, 0], [1, 1, 1]]]],
                dtype = theano.config.floatX), name ='dy')
    d45 = theano.shared(np.array([[[[-1, -1, 0], [-1, 0, 1], [0, 1, 1]]]],
                dtype = theano.config.floatX), name = 'd45')
    d135 = theano.shared(np.array([[[[0, -1, -1], [1, 0, -1], [1, 1, 0]]]],
                dtype = theano.config.floatX), name = 'd135')

    # Compute gradient of the target
    gx_img = T.nnet.conv2d(img, dx, border_mode = 'half')
    gy_img = T.nnet.conv2d(img, dy, border_mode = 'half')
    g45_img = T.nnet.conv2d(img, d45, border_mode = 'half')
    g135_img = T.nnet.conv2d(img, d135, border_mode = 'half')
    
    return gx_img, gy_img, g45_img, g135_img

def dilate_minutiae(img):
    g = theano.shared(np.array([[np.ones((5, 5))]],
                dtype = theano.config.floatX), name ='g')
    img = img.dimshuffle(0, 'x', 1, 2)
    img = T.nnet.conv2d(img, g, border_mode = 'half')
    img = T.gt(img, 0) 
    
    return img


In [ ]:
# Cost functions
from theano.tensor.signal.pool import pool_2d

## MSE encoding cost is done on an (averaged) downscaling of the image
target_pool = target
target_flat = T.flatten(target_pool, 2)
gX_pool = gX
gX_flat = T.flatten(gX_pool,2)

# Compute target minutiae based weights
#weights = T.switch(T.gt(minutiae, 0), 1, 1)
#weights = weights.flatten(2)

gx_target, gy_target, g45_target, g135_target = ridge_gradient(target_pool)
o_target, r_target, c_target = ridge_orientation(target_pool)
#o_target = pool_2d(o_target, (4,4), mode="average_exc_pad",ignore_border=True)

# Compute gradient of the output
gx_X, gy_X, g45_X, g135_X = ridge_gradient(gX_pool)
o_X, r_X, c_X = ridge_orientation(gX_pool)
#o_X = pool_2d(o_X, (4,4), mode="average_exc_pad",ignore_border=True)

enc_cost_gx = mse(gx_X.flatten(2), gx_target.flatten(2))
enc_cost_gy = mse(gy_X.flatten(2), gy_target.flatten(2))
enc_cost_g45 = mse(g45_X.flatten(2), g45_target.flatten(2))
enc_cost_g135 = mse(g135_X.flatten(2), g135_target.flatten(2))
enc_cost_o = mse(o_X.flatten(2), o_target.flatten(2))
enc_cost_r = mse(r_X.flatten(2), r_target.flatten(2))

# Final encoding cost
enc_cost_grads = (enc_cost_gx + enc_cost_gy + enc_cost_g45 + enc_cost_g135) / 4.0
enc_cost_ori = enc_cost_o / 10.0
enc_cost_rel = enc_cost_r / 10.0
enc_cost = enc_cost_grads + enc_cost_ori + enc_cost_rel
enc_cost = enc_cost.mean(dtype = theano.config.floatX)

#To change the weight of MSE, change the denominator.  ex. enc_cost/5 weights MSE much less that enc_cost/1
g_cost = enc_cost

## N.B. e_cost and e_updates will only try and minimise MSE loss on the autoencoder (for debugging)
e_cost = enc_cost

cost = [e_cost, g_cost, enc_cost_grads.mean(dtype = theano.config.floatX) , enc_cost_ori.mean(dtype = theano.config.floatX) , enc_cost_rel.mean(dtype = theano.config.floatX), enc_cost_rel.mean(dtype = theano.config.floatX)]

elrt = sharedX(0.002)
lrt = sharedX(lr)
g_updater = updates.Adam(lr=lrt, b1=b1, regularizer=updates.Regularizer(l2=l2))
e_updater = updates.Adam(lr=elrt, b1=b1, regularizer=updates.Regularizer(l2=l2))

g_updates = g_updater(e_params + g_params, g_cost)
e_updates = e_updater(e_params, e_cost)

In [ ]:
print 'COMPILING'
t = time.time()
_train_g = theano.function([X, target, minutiae], cost, updates=g_updates, allow_input_downcast = True, on_unused_input='warn')
_train_e = theano.function([X, target, minutiae], cost, updates=e_updates, allow_input_downcast = True, on_unused_input='warn')
_get_cost = theano.function([X, target, minutiae], cost, allow_input_downcast = True, on_unused_input='warn')
print '%.2f seconds to compile theano functions'%(time.time()-t)

# Testing and evaluation functions

In [ ]:
img_dir = "gen_images/"

if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    os.makedirs(os.path.join(img_dir, "tests"))
    
eval_dir = 'evals'
if not os.path.exists(os.path.join(img_dir, eval_dir)):
    os.makedirs(os.path.join(img_dir, eval_dir))

ae_encode = theano.function([X, target], [gX, target])

def inverse(X):
    X_pred = X.transpose(0, 1, 3, 2)
    X_pred = (X_pred) * 255.0
    #X_pred = (X + 1) * 127.5
    X_pred = np.rint(X_pred).astype(int)
    X_pred = np.clip(X_pred, a_min = 0, a_max = 255)
    return X_pred.astype('uint8')


def save_sample_pictures():
    for te_train, te_target, te_mask, te_minutiae in ds.train_iter():
        break
    te_out, te_ta = ae_encode(input_transform(te_train), target_transform(te_target))
    te_reshape = inverse(te_out)
    te_target_reshape = inverse(te_ta)
    te_train_reshape = inverse(input_transform(te_train))
    
    new_size = (ds.target_dim[1] * 6, ds.target_dim[0] * 12)
    new_im = Image.new('L', new_size)
    r = np.random.choice(12, 24, replace=True).reshape(2,12)
    for i in range(2):
        for j in range(12):
            index =  r[i][j]
            
            target_im = Image.fromarray(te_target_reshape[index][0])
            train_im = Image.fromarray(te_train_reshape[index][0])
            im = Image.fromarray(te_reshape[index][0])
            
            new_im.paste(target_im, (ds.target_dim[1] * i * 3, ds.target_dim[0] * j))
            new_im.paste(train_im, (ds.target_dim[1] * (i * 3 + 1), ds.target_dim[0] * j))
            new_im.paste(im, (ds.target_dim[1] * (i * 3 + 2), ds.target_dim[0] * j))
            
    img_loc = "%s/%i.png" % (img_dir, int(time.time()))    
    print "saving images to %s" %img_loc
    new_im.save(img_loc)

#saves output for all testing images.  This may take a couple of minutes to run.
def save_all_pictures():
    counter = 0
    counter2 = 0
    for te_train, te_target, te_mask, te_minutiae in ds.test_iter_all():
        te_out, te_ta = ae_encode(input_transform(te_train), target_transform(te_target))
        te_reshape = inverse(te_out)
        te_target_reshape = inverse(te_ta)
        te_train_reshape = inverse(input_transform(te_train))

        new_size = (ds.target_dim[1] * 3, ds.target_dim[0] * 12)
        new_im = Image.new('L', new_size)
        r = [range(12),range(12)]
        for i in range(1):
            for j in range(12):
                index =  r[i][j]
                try:
                    target_im = Image.fromarray(te_target_reshape[index][0])
                    train_im = Image.fromarray(te_train_reshape[index][0])
                    im = Image.fromarray(te_reshape[index][0])
                    
                    #scale = 1.6
                    #target_im.thumbnail((ds.target_dim[1] * scale, ds.target_dim[0] * scale), Image.ANTIALIAS) 
                    #train_im.thumbnail((ds.target_dim[1] * scale, ds.target_dim[0] * scale), Image.ANTIALIAS) 
                    #im.thumbnail((ds.target_dim[1] * scale, ds.target_dim[0] * scale), Image.ANTIALIAS) 
                    
                    #tgt_sz = (512, 480)
                    
                    tgt_im = Image.new('L', (ds.target_dim[1], ds.target_dim[0]))
                    tgt_im.paste(target_im, (0, 0))        
                    #tgt_im.thumbnail((ds.target_dim[1] * 2, ds.target_dim[0] * 2), Image.ANTIALIAS)            
                    img_loc = "%s/tests/test_tgt_%i.png" % (img_dir, counter2)       
                    tgt_im.save(img_loc)  

                    in_im = Image.new('L', (ds.target_dim[1], ds.target_dim[0]))
                    in_im.paste(train_im, (0, 0))                    
                    #in_im.thumbnail((ds.target_dim[1] * 2, ds.target_dim[0] * 2), Image.ANTIALIAS)
                    img_loc = "%s/tests/test_in_%i.png" % (img_dir, counter2)       
                    in_im.save(img_loc)  
                    
                    res_im = Image.new('L', (ds.target_dim[1], ds.target_dim[0]))
                    res_im.paste(im, (0, 0))
                    #res_im.thumbnail((ds.target_dim[1] * 2, ds.target_dim[0] * 2), Image.ANTIALIAS)
                    res_im.thumbnail((ds.target_dim[1] * 1.6, ds.target_dim[0] * 1.6), Image.ANTIALIAS)
                    img_loc = "%s/tests/test_pred_%i.png" % (img_dir, counter2)   
                    res_im.save(img_loc)
                    
                    counter2+=1
                    
                    new_im.paste(target_im, (ds.target_dim[1] * i * 3, ds.target_dim[0] * j))
                    new_im.paste(train_im, (ds.target_dim[1] * (i * 3 + 1), ds.target_dim[0] * j))
                    new_im.paste(im, (ds.target_dim[1] * (i * 3 + 2), ds.target_dim[0] * j))
                except:
                    print "Eror with training image"
        img_loc = "%s/test_result_%i.png" % (img_dir, counter)   
        print "saving images to %s" %img_loc
        new_im.save(img_loc)
        counter+=1
        
#saves output for all testing images.  This may take a couple of minutes to run.
def eval_all_pictures():
    counter = 0
    for te_train, te_target, te_mask, names in ds.eval_iter_all():
        te_out, te_ta = ae_encode(input_transform(te_train), target_transform(te_target))
        te_out, te_ta = ae_encode(te_out, te_ta)
        te_out, te_ta = ae_encode(te_out, te_ta)
        te_out, te_ta = ae_encode(te_out, te_ta)
        te_reshape = inverse(te_out)
        te_target_reshape = inverse(te_ta)
        te_train_reshape = inverse(input_transform(te_train))

        new_size = (ds.target_dim[1] * 3, ds.target_dim[0] * 12)
        new_im = Image.new('L', new_size)
        r = [range(12),range(12)]
        for i in range(1):
            for j in range(12):
                index =  r[i][j]
                try:
                    target_im = Image.fromarray(te_target_reshape[index][0])
                    train_im = Image.fromarray(te_train_reshape[index][0])
                    im = Image.fromarray(te_reshape[index][0])

                    tgt_im = Image.new('L', (ds.target_dim[1], ds.target_dim[0]))
                    tgt_im.paste(target_im, (0, 0))                    
                    img_loc = "%s/%s/eval_tgt_%s.png" % (img_dir, eval_dir, os.path.splitext(os.path.split(names[j])[1])[0])       
                    tgt_im.save(img_loc)  

                    in_im = Image.new('L', (ds.target_dim[1], ds.target_dim[0]))
                    in_im.paste(train_im, (0, 0))                    
                    img_loc = "%s/%s/eval_in_%s.png" % (img_dir, eval_dir, os.path.splitext(os.path.split(names[j])[1])[0])       
                    in_im.save(img_loc)  
                    
                    res_im = Image.new('L', (ds.target_dim[1], ds.target_dim[0]))
                    res_im.paste(im, (0, 0))
                    img_loc = "%s/%s/eval_pred_%s.png" % (img_dir, eval_dir, os.path.splitext(os.path.split(names[j])[1])[0])   
                    res_im.save(img_loc)
                    
                    new_im.paste(target_im, (ds.target_dim[1] * i * 3, ds.target_dim[0] * j))
                    new_im.paste(train_im, (ds.target_dim[1] * (i * 3 + 1), ds.target_dim[0] * j))
                    new_im.paste(im, (ds.target_dim[1] * (i * 3 + 2), ds.target_dim[0] * j))
                except:
                    print "Eror with training image"
        img_loc = "%s/eval_result_%i.png" % (img_dir, counter)     
        print "saving images to %s" %img_loc
        new_im.save(img_loc)
        counter+=1

#save_all_pictures()        
#save_sample_pictures()

In [ ]:
ds.epoch_size = 12
save_all_pictures()

In [ ]:
# No eval pictures available for this code example
#ds.epoch_size = 12
#eval_all_pictures()

In [ ]:
def mn(l):
    if sum(l) == 0:
        return 0
    return sum(l) / len(l)

## TODO : nicer way of coding these means?

def get_test_errors():
    print "getting test error"
    e_costs = []
    g_costs = []
    e_costs_grad = []
    e_costs_ori = []
    e_costs_rel = []
    for i in range(20):
        for samples in ds.valid_iter():
            break
        x_train, x_target, x_mask = samples
        x = input_transform(x_train)
        t = target_transform(x_target)
        m = mask_transform(x_mask)
        cost = _get_cost(x,t,m)
        e_cost, g_cost, enc_cost_grad, enc_cost_ori, enc_cost_rel, _ = cost
        e_costs.append(e_cost)
        g_costs.append(g_cost)
        e_costs_grad.append(enc_cost_grad)
        e_costs_ori.append(enc_cost_ori)
        e_costs_rel.append(enc_cost_rel)
        
    s = '[TEST] enc: %3.2f, gen: %3.2f, mse_grad: %3.2f, mse_ori: %3.2f, mse_rel: %3.2f' % (mn(e_costs), mn(g_costs), mn(e_costs_grad), mn(e_costs_ori), mn(e_costs_rel))

    return s


# Train Model

Finally, we come to the actual training of the model.  This code can be keyboard interrupted, and the weights will be stored in memory, allowing us to stop, adjust and restart the training (this is how I got the model to train).  For advice on training see the blog post at (#TODO)

In [ ]:
#iterator = tr_stream.get_epoch_iterator()

# you may wish to reset the learning rate to something of your choosing if you feel it is too high/low
lrt = sharedX(lr)

In [ ]:
theano.config.exception_verbosity = 'low'

import time
import pickle

n_updates = 0
t = time.time()

n_epochs = 350#1000

print "STARTING"

for epoch in range(301, n_epochs):
    tm = time.time()

    e_costs = []
    g_costs = []
    e_costs_grad = []
    e_costs_ori = []
    e_costs_rel = []
            
    ## TODO : produces pretty ugly output, redo this?
    for i in range(64):
        for samples in ds.train_iter():
            break
        x_train, x_target, x_mask, x_minutiae = samples
        x = input_transform(x_train)
        t = target_transform(x_target)
        #m = mask_transform(x_mask)
        m = mask_transform(x_minutiae)
        
        cost = _train_g(x,t,m) 
        e_cost, g_cost, enc_cost_grad, enc_cost_ori, enc_cost_rel, _ = cost
        e_costs.append(e_cost)
        g_costs.append(g_cost)
        e_costs_grad.append(enc_cost_grad)
        e_costs_ori.append(enc_cost_ori)
        e_costs_rel.append(enc_cost_rel)

        if n_updates % 100 == 0:
            s = '[TRAIN] enc: %3.2f, gen: %3.2f, mse_grad: %3.2f, mse_ori: %3.2f, mse_rel: %3.2f' % (mn(e_costs), mn(g_costs), mn(e_costs_grad), mn(e_costs_ori), mn(e_costs_rel))
            e_costs = []
            g_costs = []
            e_costs_grad = []
            e_costs_ori = []
            e_costs_rel = []
            logger.info(get_test_errors())
            logger.info(s)
            sys.stdout.flush()
            save_sample_pictures()
        n_updates += 1  

    logger.info("Epoch %i of %i took %.2f seconds.", epoch, n_epochs, time.time() - tm)
    
    if epoch % 50 == 0:
        all_params = [e_params, g_params]
        pickle.dump(all_params, open("model_params_%d.pkl" % epoch, 'w'))
    
    sys.stdout.flush()

# Save weights if wanted
You can reuse them by using the weights in the make_conv_set method 

In [ ]:
import pickle

all_params = [e_params, g_params, d_params]

pickle.dump(all_params, open("model_params_420.pkl", 'w'))